In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from math import radians, sin, cos, sqrt, atan2

import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse('data/ride.tcx')
root = tree.getroot()

In [3]:
data = []
for trackpoint in root.findall('.//{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Trackpoint'):
    time = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Time').text
    lat = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position/{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LatitudeDegrees').text
    lon = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position/{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LongitudeDegrees').text
    altitude = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}AltitudeMeters').text
    distance = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}DistanceMeters').text
    data.append([time, float(lat), float(lon), float(altitude), float(distance)])

df = pd.DataFrame(data, columns=['Time', 'Latitude', 'Longitude', 'AltitudeMeters', 'Total Distance (m)'])


In [4]:
df.head()

,Time,Latitude,Longitude,AltitudeMeters,Total Distance (m)
0,2023-09-22T22:41:40+00:00,37.998980,-1.13315,46.02,0.000000
1,2023-09-22T22:41:49+00:00,37.998233,-1.13288,45.98,86.408879
2,2023-09-22T22:41:59+00:00,37.997487,-1.13261,45.68,172.817758
3,2023-09-22T22:42:09+00:00,37.996740,-1.13234,43.00,259.226638
4,2023-09-22T22:42:19+00:00,37.996590,-1.13301,43.69,320.315240


### Add to DF

In [5]:
def seg_speed(row):
    if row.name == 0:
        return float('NaN')
    
    seconds = row['TimeDiff'].total_seconds()
    distance_diff = row['Total Distance (m)'] - df.loc[row.name - 1, 'Total Distance (m)']
    
    return (distance_diff / 1000) / (seconds / 3600)

In [6]:
def prepare_df(df):
    df['Time'] = pd.to_datetime(df['Time'])

    df['TimeDiff'] = df['Time'].diff()
    
    df['AltitudeChange'] = df['AltitudeMeters'].diff()
    
    df['SegmentSpeed'] = df.apply(seg_speed, axis=1)
    
    df['CumTime'] = df['TimeDiff'].cumsum()
    
    df['Total Time (m)'] = df['CumTime'].dt.total_seconds() / 60
    
    df.drop(columns=['CumTime'], inplace=True)

In [7]:
prepare_df(df)
df.head()

,Time,Latitude,Longitude,AltitudeMeters,Total Distance (m),TimeDiff,AltitudeChange,SegmentSpeed,Total Time (m)
0,2023-09-22 22:41:40+00:00,37.998980,-1.13315,46.02,0.000000,NaT,NaN,NaN,NaN
1,2023-09-22 22:41:49+00:00,37.998233,-1.13288,45.98,86.408879,0 days 00:00:09,-0.04,34.563552,0.150000
2,2023-09-22 22:41:59+00:00,37.997487,-1.13261,45.68,172.817758,0 days 00:00:10,-0.30,31.107196,0.316667
3,2023-09-22 22:42:09+00:00,37.996740,-1.13234,43.00,259.226638,0 days 00:00:10,-2.68,31.107197,0.483333
4,2023-09-22 22:42:19+00:00,37.996590,-1.13301,43.69,320.315240,0 days 00:00:10,0.69,21.991897,0.650000


### Functions

In [8]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

In [9]:
def calc_total_distance(df):
    total_distance = 0
    for i in range(1, len(df)):
        lat1, lon1 = df['Latitude'][i - 1], df['Longitude'][i - 1]
        lat2, lon2 = df['Latitude'][i], df['Longitude'][i]

        seg_distance = haversine(lat1, lon1, lat2, lon2)
        total_distance += seg_distance
        
    return total_distance

In [10]:
def calc_moving_time(df):
    return df['TimeDiff'].sum()

In [11]:
def elevation_info(df):
    total_ascent = df['AltitudeChange'][df['AltitudeChange'] > 0].sum()
    total_descent = df['AltitudeChange'][df['AltitudeChange'] < 0].sum()
    total_change = df['AltitudeChange'].sum()
    lowest = df['AltitudeMeters'].min()
    highest = df['AltitudeMeters'].max()
    
    return total_ascent, total_descent, total_change, lowest, highest

In [12]:
def speed_info(df):
    average = calc_total_distance(df) / (calc_moving_time(df).total_seconds() / 3600)
    fastest = df['SegmentSpeed'].max()
    slowest = df['SegmentSpeed'].min()
    
    return average, fastest, slowest

### Extractions

In [13]:
f'The total distance is {calc_total_distance(df)} km/h.'

'The total distance is 84.79457109912336 km/h.'

In [14]:
f'The total moving time is {calc_moving_time(df)}'

'The total moving time is 0 days 04:04:31'

In [15]:
f'The total ascent was {elevation_info(df)[0]} metres'

'The total ascent was 476.24 metres'

In [16]:
f'The total descent was {elevation_info(df)[1]} metres'

'The total descent was -519.77 metres'

In [17]:
f'The total change was {elevation_info(df)[2]} metres'

'The total change was -43.53000000000003 metres'

In [18]:
f'The lowest altitude was {elevation_info(df)[3]} metres'

'The lowest altitude was 2.4899999999999998 metres'

In [19]:
f'The highest altitude was {elevation_info(df)[4]} metres'

'The highest altitude was 248.85000000000002 metres'

In [20]:
f'The average speed was {speed_info(df)[0]} km/h'

'The average speed was 20.807065364109064 km/h'

In [21]:
f'The fastest speed was {speed_info(df)[1]} km/h'

'The fastest speed was 39.98372614939071 km/h'

In [22]:
f'The slowest speed was {speed_info(df)[2]} km/h'

'The slowest speed was 1.1349515521869762 km/h'

### Visualising

In [23]:
import plotly.express as px

In [28]:
import plotly.express as px
import plotly.graph_objs as go

# Your DataFrame preparation code here

# Create the line plot
fig = px.line(df, x='Total Time (m)', y='Total Distance (m)', title='Distance Over Time')

# Customize the plot further (optional)
fig.update_xaxes(title_text='Time (minutes)')
fig.update_yaxes(title_text='Distance (Km)')

# Add a filled area under the curve
fig.add_trace(go.Scatter(x=df['Total Time (m)'], y=df['Total Distance (m)'], fill='tozeroy', fillcolor='rgba(0,100,80,0.2)'))

# Show the plot
fig.show()